In [ ]:
#MODULOS DE SELENIUM
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service



from datetime import datetime

#MODULOS SECUNDARIOS
import pandas as pd
import numpy as np
import pyautogui
import os
import time
import ssl
import sys
import traceback

#MODULO ALERTAS DESACTIVADAS
import warnings
warnings.filterwarnings('ignore')

#MODULO PROPIOS
from utils import modelamiento


In [ ]:
URL_POST  = pd.read_excel('url/dataset_210425_1.xlsx') 
lista_url = list(URL_POST['url'])

# URL_POST  = pd.read_excel(r'instagram scraper_apify/160325_posts.xlsx').fillna(0) 
# URL_POST  = URL_POST[URL_POST['latestComments/0/id'] != 0]
# lista_url = list(URL_POST['url'])

In [ ]:
def ManagerInstagramScrapper (url,medio):

    # # Configuracion Chrome
    # chrome_options = Options()
    # # chrome_options.add_argument(f"--user-agent={random_user_agent}")
    # chrome_options.add_argument("--incognito")  # Modo incógnito
    # chrome_options.add_argument("--disable-cache")  # Deshabilitar la caché
    # chrome_options.add_argument("--disable-application-cache")  # Deshabilitar caché de aplicaciones
    # chrome_options.add_argument("--disable-gpu")  # Deshabilitar GPU (opcional en algunos casos)
    # chrome_options.add_argument("--disable-dev-shm-usage")  # Para evitar problemas en entornos Linux
    # chrome_options.add_argument("--no-sandbox")  # Para evitar problemas de permisos en Linux
    # chrome_options.add_argument("--disable-features=NetworkService,NetworkServiceInProcess")  # Más control sobre caché

    # chrome_options.add_argument("--start-maximized")  # Abre la ventana en pantalla completa
    # # Crear la instancia del WebDriver con las opciones configuradas
    # driver = webdriver.Chrome(options=chrome_options)
    # driver.maximize_window()

    # # Move mouse with a smooth transition

    # driver.get("https://www.instagram.com/")




    # Configuración de Firefox
    firefox_options = Options()
    # firefox_options.set_preference("general.useragent.override", random_user_agent)  # Si usas user-agent aleatorio

    firefox_options.set_preference("browser.privatebrowsing.autostart", True)  # Modo incógnito
    firefox_options.set_preference("browser.cache.disk.enable", False)  # Desactiva caché en disco
    firefox_options.set_preference("browser.cache.memory.enable", False)  # Desactiva caché en memoria
    firefox_options.set_preference("browser.cache.offline.enable", False)  # Desactiva caché offline
    firefox_options.set_preference("network.http.use-cache", False)  # Desactiva caché HTTP

    # Deshabilita aceleración por hardware
    firefox_options.set_preference("layers.acceleration.disabled", True)

    # Evita problemas de memoria compartida y sandbox (no tan necesario en Firefox, pero por coherencia)
    firefox_options.set_preference("security.sandbox.content.level", 0)

    # Ventana maximizada
    firefox_options.add_argument("--start-maximized")

    # Instanciar WebDriver
    driver = webdriver.Firefox(options=firefox_options)
    driver.maximize_window()

    # Move mouse with a smooth transition

    driver.get("https://www.instagram.com/")

    pyautogui.moveTo(1645, 500, duration=0.5)  # Moves in 1.5 seconds
    for _ in range(5):  # Repeat 4 times
        pyautogui.click() 
        pyautogui.scroll(-800)
        pyautogui.scroll(300)  # Scroll up by 500
        time.sleep(1)  # Short delay to mimic human behavior


    # driver = webdriver.Firefox()

    driver.get(url)
    time.sleep(8)
    

    try:
        time.sleep(1)
        pyautogui.moveTo(1250, 300, duration=1)
        pyautogui.click()
    
    except:
        time.sleep(1)
        # Espera a que el botón sea clickeable
        boton  = driver.find_element(By.XPATH, value = "/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div[1]/div").click()                                                                
    
        
    finally:
        pass

    
# /html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[1]/div/div 
# "div.x1lliihq.x1n2onr6.x1roi4f4"

    pyautogui.moveTo(1250, 463, duration=0.5) 
    for _ in range(8):  # Repeat 4 times
        # pyautogui.click() 
        pyautogui.scroll(-300)  # Scroll up by 500
        time.sleep(2.5) 
    
    
    # driver.get(url)
    # time.sleep(15)    

    comentarios = driver.find_elements(By.CSS_SELECTOR,"div._a9zr") #CLASE DEL CONTENEDOR DE NOMBRE DE USUARIO, COMENTARIO, HORA Y REACCIONES

    # Extraer los textos de los comentarios

    textos_comentarios = [comentario.text for comentario in comentarios]
    procesados         = [modelamiento(texto) for texto in textos_comentarios]
    flattened_data = [item for sublist in procesados for item in sublist]

    # Crear un DataFrame dividiendo cada elemento en "Usuario" y "Comentario"

    df = pd.DataFrame(flattened_data, columns=["Usuario", "Comentario", "Hora", "Reacciones"])
    df["url_instagram"] = url
    # df = pd.DataFrame([comentario.split('\n', 1) for comentario in textos_comentarios], columns=['Usuario', 'Comentario'])

    # Guardar en un archivo Excel

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f'data/data_{medio}_{timestamp}.xlsx'
    # filename = f'data_{medio}_{timestamp}.xlsx'

    df.to_excel(filename, index=False)
    # print(textos_comentarios)
    driver.quit()
    print("Excel generado con éxito 🎉")    

In [ ]:
for url in lista_url:
    ManagerInstagramScrapper(url,medio="medios")
    

In [10]:
# LIMPIEZA DATAFRAME Y CONSOLIDADO DIARIO
ruta_directorio = "/data/data final"
ruta_archivo = os.path.join(ruta_directorio, "archivo_unido.xlsx")

carpeta = '/Users/admin/Documents/Documentos El Barometro/GitHub/BX-Codes/01 Instagram - Proyectos/instagram scraper/data/230425'

archivos = [f for f in os.listdir(carpeta) if f.endswith((".xlsx",".xls"))]
dataframes = []

for archivo in archivos:
    ruta_completa = os.path.join(carpeta,archivo)
    df = pd.read_excel(ruta_completa)
    dataframes.append(df)

df_final = pd.concat(dataframes,ignore_index=True)




In [11]:
df_final.to_excel('data_facebook_bx.xlsx',index=True)

In [ ]:
ecuador = pd.read_excel('/Users/admin/Documents/Documentos El Barometro/GitHub/BX-Codes/01 Instagram - Proyectos/instagram scraper/data/140425/ecuador_pre_140425.xlsx')
ecuador